# Importing Libraries & Reading JSON

In [4]:
import sys
import os
import subprocess
import json
import itertools

json_list = json.load(open('parameters_json_file.json', 'r'))
print("parameters_json_file.json just opened")

data = json_list["data"]
parameters = json_list["parameters"]
fixed_parameters = parameters["fixed"]
iterable_parameters = parameters["iterable"]
print(iterable_parameters)

parameters_json_file.json just opened
{'rnn_dim': [100, 150, 200, 250, 300], 'char_rnn': [25, 50], 'word_min_freq': [0, 2, 4, 8, 16], 'train_batch_size': [4, 8, 16, 32, 64]}


## Parameters Combinations

In [5]:
listofvalues=iterable_parameters.values()
args_list = []
for it_key in iterable_parameters:
    print(it_key, "\t", iterable_parameters[it_key])
    args_list.append(it_key)
parameter_combinations = list(itertools.product(*listofvalues))
print("\nNumber of combinations to try:", len(parameter_combinations))

rnn_dim 	 [100, 150, 200, 250, 300]
char_rnn 	 [25, 50]
word_min_freq 	 [0, 2, 4, 8, 16]
train_batch_size 	 [4, 8, 16, 32, 64]

Number of combinations to try: 250


In [16]:
index_file = open("index.txt", "w")
for comb in parameter_combinations:
    string = str(parameter_combinations.index(comb))
    for num in comb:
        string += "\t" + str(num)
    string += "\n"
    index_file.write(string)
index_file.close()
print(string)

249	300	50	16	64



## Class: Command - add_arg()

In [18]:
class Command_string:
    def __init__(self, initial_string = "python"):
        self.string = initial_string
    
    def change(self, new_string):
        self.string = new_string
    
    def add_arg_str(self, string):
        self.string += " " + string
        
    def add_arg_dic(self, key, dic):
        self.string += " --" + key + " " + str(dic[key])
    
    def add_arg_val(self, key, val):
        self.string += " --" + key + " " + str(val)

## Commands String

### Fixed string part

In [19]:
fixed_command = Command_string()
fixed_command.add_arg_str(json_list["train.py"])
for path_key in data:
    fixed_command.add_arg_dic(path_key, data)
for fixed_par_key in fixed_parameters:
    fixed_command.add_arg_dic(fixed_par_key, fixed_parameters)
print(fixed_command.string)

python ~/work/model/train.py --data_path ~/work/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path ~/work/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 64 --val_batch_size 64 --max_epochs 100


### Changing common part (due to arguments combinations) --> list of strings

In [20]:
common_list = []
tmp_command = Command_string()
name = []
checkpoint_dirs = []

for comb in parameter_combinations:
    tmp_command.change(fixed_command.string)
    comb_num = parameter_combinations.index(comb)
    name.append("training_" + str(comb_num) + "_")
    for it_par_key in iterable_parameters:
        key_index = list(iterable_parameters.keys()).index(it_par_key)
        tmp_command.add_arg_val(it_par_key, comb[key_index])             # adds --parameterName parameterValue
        name[comb_num] += "__" + it_par_key + "_" + str(comb[key_index]) # adds __parameterName_parameterValue
    common_list.append(tmp_command.string)

### Showing one command

In [21]:
print("List length:", len(common_list))
index_to_show = 3
print("List command %d (%d/%d):\n%s" % (index_to_show, index_to_show+1, len(common_list), common_list[index_to_show]))

List length: 250
List command 3 (4/250):
python ~/work/model/train.py --data_path ~/work/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path ~/work/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 64 --val_batch_size 64 --max_epochs 100 --rnn_dim 100 --char_rnn 25 --word_min_freq 0 --train_batch_size 32


## Training command

In [22]:
train_command = Command_string()
train_list = common_list.copy()
for comb in parameter_combinations:
    train_command.change("")
    comb_num = parameter_combinations.index(comb)
    train_command.add_arg_val("checkpoint", json_list["checkpoints"] + "/" + name[comb_num])
    train_command.add_arg_str(" |& sed 's/val acc/\\nval acc/g' | grep 'val acc\|test acc\|Epoch' | tee -a " \
                              + json_list["logs"] + "/" + name[comb_num] + ".log")
    train_list[comb_num] += train_command.string

In [23]:
print(train_command.string)
print("List length:", len(train_list))
index_to_show = 71
print("List command %d (%d/%d):\n%s" % (index_to_show, index_to_show+1, len(train_list), train_list[index_to_show]))


print("List command %d (%d/%d):\n%s" % (index_to_show, index_to_show+1, len(common_list), common_list[index_to_show]))


 --checkpoint ~/work/trainings/training_2/checkpoints/training_249___rnn_dim_300__char_rnn_50__word_min_freq_16__train_batch_size_64  |& sed 's/val acc/\nval acc/g' | grep 'val acc\|test acc\|Epoch' | tee -a ~/work/trainings/training_2/logs/training_249___rnn_dim_300__char_rnn_50__word_min_freq_16__train_batch_size_64.log
List length: 250
List command 71 (72/250):
python ~/work/model/train.py --data_path ~/work/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path ~/work/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 64 --val_batch_size 64 --max_epochs 100 --rnn_dim 150 --char_rnn 25 --word_min_freq 16 --train_batch_size 8 --checkpoint ~/work/trainings/training_2/checkpoints/training_71___rnn_dim_150__char_rnn_25__word_min_freq_16__train_batch_size_8  |& sed 's/val acc/\nval acc/g' | grep 'val acc\|test acc\|Epoch' | tee -a ~/work/trainings/training_2/logs/training_71___rnn_dim_150__char_rnn_25__word_min_fr

# Output Files

## Train File

In [24]:
#Opening and writing in train file
for gpu in range(0,4):
    file_name = json_list["commands_dir"] + "CUDA_" + str(gpu) + "_" + json_list["output_train"]
    train_file = open(file_name, 'w')
    for line in train_list:
        train_file.write('CUDA_VISIBLE_DEVICES="' + str(gpu) + '" ' +line + "\n")
    train_file.close()

In [25]:
gpu = 0
file_name = json_list["commands_dir"] + "CUDA_" + str(gpu) + "_" + json_list["output_train"]
train_file = open(file_name, 'r')
print("Train file:", file_name, "\n\n", train_file.read())
train_file.close()

Train file: CUDA_0_train_2.cmd 

 CUDA_VISIBLE_DEVICES="0" python ~/work/model/train.py --data_path ~/work/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path ~/work/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 64 --val_batch_size 64 --max_epochs 100 --rnn_dim 100 --char_rnn 25 --word_min_freq 0 --train_batch_size 4 --checkpoint ~/work/trainings/training_2/checkpoints/training_0___rnn_dim_100__char_rnn_25__word_min_freq_0__train_batch_size_4  |& sed 's/val acc/\nval acc/g' | grep 'val acc\|test acc\|Epoch' | tee -a ~/work/trainings/training_2/logs/training_0___rnn_dim_100__char_rnn_25__word_min_freq_0__train_batch_size_4.log
CUDA_VISIBLE_DEVICES="0" python ~/work/model/train.py --data_path ~/work/data/CRF --train_path train.bilou_CRF --val_path validation.bilou_CRF --test_path test.bilou_CRF --emb_path ~/work/data/embedding/pretrained_embedding.txt --char_emb_dim 25 --word_emb_dim 64 --val_batch_size 64

## Test file

## Test command